In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np

In [24]:
transform = transforms.Compose(
    [
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

batch_size = 100

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [25]:
class ConvolutionalNet(nn.Module):
    def __init__(self):
        super(ConvolutionalNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(256 *4 *4, 512)
        self.fc2 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 256 * 4 * 4)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

In [26]:
"""class ConvolutionalNet(nn.Module):
    def __init__(self):
        super(ConvolutionalNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(512 *2 *2, 512)
        self.fc2 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = self.pool(torch.relu(self.conv4(x)))
        x = x.view(-1, 512 * 2 * 2)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x"""

'class ConvolutionalNet(nn.Module):\n    def __init__(self):\n        super(ConvolutionalNet, self).__init__()\n        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)\n        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)\n        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)\n        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)\n        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)\n        self.fc1 = nn.Linear(512 *2 *2, 512)\n        self.fc2 = nn.Linear(512, 10)\n        self.dropout = nn.Dropout(p=0.5)\n\n    def forward(self, x):\n        x = self.pool(torch.relu(self.conv1(x)))\n        x = self.pool(torch.relu(self.conv2(x)))\n        x = self.pool(torch.relu(self.conv3(x)))\n        x = self.pool(torch.relu(self.conv4(x)))\n        x = x.view(-1, 512 * 2 * 2)\n        x = torch.relu(self.fc1(x))\n        x = self.dropout(x)\n        x = self.fc2(x)\n        \n        return x'

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ConvolutionalNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs =  5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    correct = 0
    total = 0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()            # Zero the parameter gradients
        outputs = model(images)           # Forward pass
        loss = criterion(outputs, labels) # Compute loss
        loss.backward()                   # Backward pass
        optimizer.step()                  # Update weights
            
        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(trainloader):.4f} Accuracy: {(correct*100) / total} %")


Epoch 1/5, Loss: 1.4402 Accuracy: 47.504 %
Epoch 2/5, Loss: 0.9979 Accuracy: 64.82 %
Epoch 3/5, Loss: 0.7977 Accuracy: 72.086 %
Epoch 4/5, Loss: 0.6693 Accuracy: 76.446 %
Epoch 5/5, Loss: 0.5688 Accuracy: 80.036 %


In [28]:
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
class_correct = [0] * 10  # List to store correct predictions for each class
class_total = [0] * 10    # List to store total predictions for each class

with torch.no_grad():  # Disable gradient calculation
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)  
        total += labels.size(0)
        correct += (predicted == labels).sum().item()  # Count correct predictions
        c = (predicted == labels)
       
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

# Calculate and print accuracy
accuracy = 100 * correct / total
print(f"Test Accuracy after epoch {epoch + 1}: {accuracy:.2f}% \n")

for i in range(10):
    accuracy = 100 * class_correct[i] / class_total[i]
    print(f"Accuracy of class {i + 1}: {accuracy:.2f}%")

Test Accuracy after epoch 5: 76.56% 

Accuracy of class 1: 80.20%
Accuracy of class 2: 92.10%
Accuracy of class 3: 68.90%
Accuracy of class 4: 64.30%
Accuracy of class 5: 73.70%
Accuracy of class 6: 62.40%
Accuracy of class 7: 83.80%
Accuracy of class 8: 78.50%
Accuracy of class 9: 87.30%
Accuracy of class 10: 74.40%
